# Hotel Reviews NLP - Exploratory Data Analysis

This notebook explores the 515K Hotel Reviews Dataset from Kaggle.

**Dataset**: https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe

## Key Questions:
1. What are the most common complaints and praises?
2. How do review patterns differ by reviewer nationality?
3. What text features correlate with review scores?
4. Are there seasonal or temporal patterns?

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import ast

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

%matplotlib inline

## 1. Load and Inspect Data

In [ ]:
# Load data - update path as needed
df = pd.read_csv('../data/Hotel_Reviews.csv')  # Full dataset
# df = pd.read_csv('../data/Hotel_Reviews_sample.csv')  # Sample for testing

print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.head()

In [ ]:
# Data types and missing values
info_df = pd.DataFrame({
    'dtype': df.dtypes,
    'non_null': df.count(),
    'null_count': df.isnull().sum(),
    'null_pct': (df.isnull().sum() / len(df) * 100).round(2)
})
info_df

In [ ]:
# Numerical summary
df.describe().round(2)

## 2. Score Distribution Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
df['Reviewer_Score'].hist(bins=20, ax=ax1, color='steelblue', edgecolor='white')
ax1.axvline(df['Reviewer_Score'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df["Reviewer_Score"].mean():.2f}')
ax1.axvline(df['Reviewer_Score'].median(), color='orange', linestyle='--', 
            label=f'Median: {df["Reviewer_Score"].median():.2f}')
ax1.set_xlabel('Reviewer Score')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Reviewer Scores')
ax1.legend()

# Box plot by score category
ax2 = axes[1]
score_bins = pd.cut(df['Reviewer_Score'], bins=[0, 4, 6, 8, 9, 10], 
                    labels=['1-4', '4-6', '6-8', '8-9', '9-10'])
score_dist = score_bins.value_counts().sort_index()
score_dist.plot(kind='bar', ax=ax2, color='steelblue', edgecolor='white')
ax2.set_xlabel('Score Range')
ax2.set_ylabel('Count')
ax2.set_title('Reviews by Score Category')
ax2.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('../outputs/score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Hotel Analysis

In [ ]:
print(f"Unique Hotels: {df['Hotel_Name'].nunique()}")
print(f"Unique Countries (Reviewers): {df['Reviewer_Nationality'].nunique()}")

# Top hotels
top_hotels = df.groupby('Hotel_Name').agg({
    'Reviewer_Score': ['count', 'mean', 'std']
}).round(2)
top_hotels.columns = ['review_count', 'avg_score', 'std_score']
top_hotels = top_hotels.sort_values('review_count', ascending=False).head(15)
top_hotels

In [ ]:
# Score distribution by top 10 hotels
fig, ax = plt.subplots(figsize=(12, 8))

top_10_hotels = df['Hotel_Name'].value_counts().head(10).index
df_top10 = df[df['Hotel_Name'].isin(top_10_hotels)]
hotel_order = df_top10.groupby('Hotel_Name')['Reviewer_Score'].mean().sort_values(ascending=False).index

sns.boxplot(data=df_top10, y='Hotel_Name', x='Reviewer_Score', order=hotel_order)
ax.set_title('Score Distribution by Hotel (Top 10 by Volume)')
ax.set_xlabel('Reviewer Score')
ax.set_ylabel('')

plt.tight_layout()
plt.savefig('../outputs/hotel_scores_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Reviewer Nationality Analysis

In [ ]:
# Clean nationality and get stats
df['Nationality_Clean'] = df['Reviewer_Nationality'].str.strip()

nationality_stats = df.groupby('Nationality_Clean').agg({
    'Reviewer_Score': ['count', 'mean']
}).round(2)
nationality_stats.columns = ['count', 'avg_score']
nationality_stats = nationality_stats.sort_values('count', ascending=False)

# Top 15
nationality_stats.head(15)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
ax1 = axes[0]
top_nations = nationality_stats.head(10)
ax1.pie(top_nations['count'], labels=top_nations.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Top 10 Reviewer Nationalities')

# Average score by nationality
ax2 = axes[1]
top_nations_sorted = top_nations.sort_values('avg_score')
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(top_nations_sorted)))
ax2.barh(top_nations_sorted.index, top_nations_sorted['avg_score'], color=colors)
ax2.set_xlim(7.5, 9)
ax2.set_xlabel('Average Score')
ax2.set_title('Average Score by Nationality (Top 10)')

plt.tight_layout()
plt.savefig('../outputs/nationality_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Text Analysis

In [ ]:
# Check for placeholder reviews
no_negative = (df['Negative_Review'].str.strip().str.lower() == 'no negative').sum()
no_positive = (df['Positive_Review'].str.strip().str.lower() == 'no positive').sum()

print(f"'No Negative' placeholder reviews: {no_negative:,} ({no_negative/len(df)*100:.1f}%)")
print(f"'No Positive' placeholder reviews: {no_positive:,} ({no_positive/len(df)*100:.1f}%)")
print(f"\nNegative Review Length - Mean: {df['Review_Total_Negative_Word_Counts'].mean():.1f}, Median: {df['Review_Total_Negative_Word_Counts'].median():.1f}")
print(f"Positive Review Length - Mean: {df['Review_Total_Positive_Word_Counts'].mean():.1f}, Median: {df['Review_Total_Positive_Word_Counts'].median():.1f}")

In [ ]:
# Word frequency analysis
def get_word_freq(text_series, top_n=20):
    """Extract word frequencies from text, removing stopwords."""
    all_text = ' '.join(text_series.astype(str).str.lower())
    words = re.findall(r'\b[a-z]{3,}\b', all_text)
    
    stopwords = {'the', 'and', 'was', 'for', 'that', 'with', 'our', 'were', 'had', 'but', 
                 'are', 'not', 'have', 'this', 'from', 'they', 'would', 'there', 'been',
                 'which', 'could', 'when', 'about', 'very', 'just', 'also', 'only', 'more',
                 'some', 'than', 'what', 'into', 'out', 'all', 'you', 'can', 'one', 'get',
                 'hotel', 'room', 'stay', 'stayed', 'night', 'nights', 'did', 'didn', 'don'}
    
    filtered_words = [w for w in words if w not in stopwords]
    return Counter(filtered_words).most_common(top_n)

# Filter out placeholder reviews
real_negative = df[df['Negative_Review'].str.strip().str.lower() != 'no negative']['Negative_Review']
real_positive = df[df['Positive_Review'].str.strip().str.lower() != 'no positive']['Positive_Review']

neg_freq = get_word_freq(real_negative, 20)
pos_freq = get_word_freq(real_positive, 20)

print("Top 20 Negative Review Words:")
for word, count in neg_freq:
    print(f"  {word}: {count:,}")

In [ ]:
# Visualize word frequencies
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Negative
ax1 = axes[0]
words, counts = zip(*neg_freq)
ax1.barh(range(len(words)), counts, color='salmon', edgecolor='white')
ax1.set_yticks(range(len(words)))
ax1.set_yticklabels(words)
ax1.invert_yaxis()
ax1.set_xlabel('Frequency')
ax1.set_title('Top 20 Words in Negative Reviews', fontweight='bold')

# Positive
ax2 = axes[1]
words, counts = zip(*pos_freq)
ax2.barh(range(len(words)), counts, color='lightgreen', edgecolor='white')
ax2.set_yticks(range(len(words)))
ax2.set_yticklabels(words)
ax2.invert_yaxis()
ax2.set_xlabel('Frequency')
ax2.set_title('Top 20 Words in Positive Reviews', fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/word_frequency.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Tags Analysis

In [ ]:
# Parse tags
def parse_tags(tag_str):
    try:
        tags = ast.literal_eval(tag_str)
        return [t.strip().strip("'") for t in tags]
    except:
        return []

df['parsed_tags'] = df['Tags'].apply(parse_tags)
all_tags = [tag for tags in df['parsed_tags'] for tag in tags]
tag_counts = Counter(all_tags).most_common(20)

print("Top 20 Tags:")
for tag, count in tag_counts:
    print(f"  {tag}: {count:,} ({count/len(df)*100:.1f}%)")

In [ ]:
# Score by trip type
df['is_leisure'] = df['Tags'].str.contains('Leisure', case=False)
df['is_business'] = df['Tags'].str.contains('Business', case=False)

print(f"Leisure trips: {df['is_leisure'].sum():,} ({df['is_leisure'].mean()*100:.1f}%)")
print(f"Business trips: {df['is_business'].sum():,} ({df['is_business'].mean()*100:.1f}%)")
print(f"\nAverage Score - Leisure: {df[df['is_leisure']]['Reviewer_Score'].mean():.2f}")
print(f"Average Score - Business: {df[df['is_business']]['Reviewer_Score'].mean():.2f}")

## 7. Temporal Analysis

In [ ]:
# Parse dates
df['Review_Date'] = pd.to_datetime(df['Review_Date'], format='%m/%d/%Y')
df['year'] = df['Review_Date'].dt.year
df['month'] = df['Review_Date'].dt.month
df['month_name'] = df['Review_Date'].dt.month_name()
df['day_of_week'] = df['Review_Date'].dt.day_name()

print(f"Date Range: {df['Review_Date'].min().date()} to {df['Review_Date'].max().date()}")

In [ ]:
# Monthly trend
monthly = df.groupby(df['Review_Date'].dt.to_period('M')).agg({
    'Reviewer_Score': ['count', 'mean']
}).reset_index()
monthly.columns = ['month', 'count', 'avg_score']
monthly['month'] = monthly['month'].dt.to_timestamp()

fig, ax1 = plt.subplots(figsize=(14, 6))
ax2 = ax1.twinx()

ax1.bar(monthly['month'], monthly['count'], alpha=0.4, color='steelblue', label='Review Count', width=20)
ax2.plot(monthly['month'], monthly['avg_score'], 'r-o', markersize=4, linewidth=2, label='Avg Score')

ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Reviews', color='steelblue')
ax2.set_ylabel('Average Score', color='red')
ax1.set_title('Monthly Review Volume & Average Score')

plt.tight_layout()
plt.savefig('../outputs/temporal_trend.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Correlation Analysis

In [ ]:
# Key correlations
print("Correlations with Reviewer Score:")
print(f"  Negative word count: {df['Review_Total_Negative_Word_Counts'].corr(df['Reviewer_Score']):.3f}")
print(f"  Positive word count: {df['Review_Total_Positive_Word_Counts'].corr(df['Reviewer_Score']):.3f}")
print(f"  Hotel average score: {df['Average_Score'].corr(df['Reviewer_Score']):.3f}")
print(f"  Reviewer experience: {df['Total_Number_of_Reviews_Reviewer_Has_Given'].corr(df['Reviewer_Score']):.3f}")

In [ ]:
# Correlation heatmap
numeric_cols = ['Average_Score', 'Review_Total_Negative_Word_Counts', 
                'Review_Total_Positive_Word_Counts', 'Reviewer_Score',
                'Total_Number_of_Reviews_Reviewer_Has_Given']

fig, ax = plt.subplots(figsize=(10, 8))
corr = df[numeric_cols].corr()
short_names = ['Hotel Avg', 'Neg Words', 'Pos Words', 'Score', 'Reviewer Exp']
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm', center=0,
            xticklabels=short_names, yticklabels=short_names, fmt='.2f')
ax.set_title('Correlation Matrix')

plt.tight_layout()
plt.savefig('../outputs/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Key Findings Summary

### Score Distribution
- Scores are **negatively skewed** - most reviews are positive (mean ~8.4, median ~8.8)
- ~49% of reviews score 9-10

### Text Patterns
- **~24% of negative reviews** are placeholders ("No Negative")
- **~8% of positive reviews** are placeholders
- **Negative word count strongly correlates with lower scores** (r = -0.39)
- Common complaints: small rooms, breakfast quality, noise
- Common praises: staff, location, cleanliness

### Demographics
- UK reviewers dominate (~63%)
- Leisure travelers give higher scores than business travelers
- Couples and groups rate highest; solo travelers rate lowest

### Next Steps for NLP
1. **Sentiment Analysis**: Validate scores against text sentiment
2. **Topic Modeling**: Extract themes from positive/negative reviews
3. **Score Prediction**: Build classifier using text features

In [ ]:
# Save cleaned data for next phase
# df.to_parquet('../data/hotel_reviews_processed.parquet', index=False)
print("EDA Complete! ✅")